<a href="https://colab.research.google.com/github/zacbakerr/open-fema-analysis/blob/main/fema_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

In [2]:
# for conversion between datasets that use different notation

state_abbreviations = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming'
}

def get_state_name(abbreviation):
    return state_abbreviations.get(abbreviation.upper(), 'Unknown')

In [3]:
# map each disaster number to the type of disaster it is
disaster_types = dict()
declarations = pd.read_csv("drive/MyDrive/Research/FEMA/data/DisasterDeclarationsSummaries.csv")
for index, disasterNumer in enumerate(declarations["disasterNumber"]):
  disaster_types[disasterNumer] = str(declarations["incidentType"][index])

# map each county to the percent of white people it has
county_to_white_percentage = dict()
race = pd.read_csv("drive/MyDrive/Research/FEMA/data/race.csv")
for index, county in enumerate(race["NAME"]):
  if "Geographic Area Name" in county: continue
  split = county.split(", ")
  if "County" in split[0]: split[0] = split[0].replace(" County", "")
  county_to_white_percentage[split[0] + ", " + split[1]] = (int(race["P1_003N"][index]) / int(race["P1_001N"][index])) * 100

# map each county to its median income
county_to_income = dict()
census = pd.read_csv("drive/MyDrive/Research/FEMA/data/income.csv")
for index, county in enumerate(census["NAME"]):
  if "Geographic Area Name" in county: continue
  split = county.split(", ")
  if "County" in split[0]: split[0] = split[0].replace(" County", "")
  # some counties dont have income data
  if census["S1901_C01_012E"][index] == "-": county_to_income[split[0] + ", " + split[1]] = 30,000
  else: county_to_income[split[0] + ", " + split[1]] = int(census["S1901_C01_012E"][index])

# map each county to population
county_to_population = dict()
census = pd.read_csv("drive/MyDrive/Research/FEMA/data/vulnerability.csv")
for index, county in enumerate(census["NAME"]):
  if "Geographic Area Name" in county: continue
  split = county.split(", ")
  if "County" in split[0]: split[0] = split[0].replace(" County", "")
  # some counties dont have income data
  county_to_population[split[0] + ", " + split[1]] = int(census["POPUNI"][index])

In [15]:
# load and filter data

owners_data = pd.read_csv('drive/MyDrive/Research/FEMA/data/HousingAssistanceOwners.csv', delimiter=',')
owners_data.columns = owners_data.columns.str.strip()

states = ["FL", "NC", "SC"]
key_disaster_types = ["FLood", "Hurricane"]

filtered_data = owners_data[owners_data["state"].isin(states)]
filtered_data = filtered_data[filtered_data["disasterNumber"].map(disaster_types).isin(key_disaster_types)]

# create dictionary of total county damage

total_county_damage = dict()
for index, row in filtered_data.iterrows():
  if row["county"] + ", " + row["state"] not in total_county_damage:
    total_county_damage[row["county"] + ", " + row["state"]] = float(row["totalDamage"])
  else:
    total_county_damage[row["county"] + ", " + row["state"]] += float(row["totalDamage"])

# find the county with the max damage
max(total_county_damage, key=lambda k: total_county_damage[k])

<ipython-input-15-92675b8bd3a4>:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  owners_data = pd.read_csv('drive/MyDrive/Research/FEMA/data/HousingAssistanceOwners.csv', delimiter=',')


'Lee (County), FL'

In [5]:
# read in FEMA Individual Assistance Dataset (140,249 rows)
owners_data = pd.read_csv('drive/MyDrive/Research/FEMA/data/HousingAssistanceOwners.csv', delimiter=',')
owners_data.columns = owners_data.columns.str.strip()
data = owners_data

# add income and white percentage as columns to our filtered data
data['county_state'] = data.apply(lambda row: row["county"].replace(" (County)", "") + ", " + get_state_name(row["state"]), axis=1)

# Add additional columns for white percentage and median income
data['white_percentage'] = data['county_state'].map(county_to_white_percentage)
data['median_income'] = data['county_state'].map(county_to_income)
data['population'] = data['county_state'].map(county_to_population)

# look at southern coast: TX, LA, MS, AL, FL (38,897 rows)
entriesPerState = data["state"].value_counts()
filtered_data = data[data['state'].isin(["TX", "MS", "AL", "GA", "FL", "LA"])]

# look at floods and hurricans (26,365 rows)
filtered_data = filtered_data[
    filtered_data["disasterNumber"].map(disaster_types).isin(["Flood", "Hurricane"])
]

# look at disasters where averageFemaInspectedDamage was above $1,500 (6,156  rows) (4.4% of original row count)
filtered_data = filtered_data[filtered_data['averageFemaInspectedDamage'] > 1500]

<ipython-input-5-66821258db7c>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  owners_data = pd.read_csv('drive/MyDrive/Research/FEMA/data/HousingAssistanceOwners.csv', delimiter=',')
